In [1]:
%config IPCompleter.greedy=True

In [2]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

'1.0.1'

In [3]:
url = 'https://www.minsal.cl/nuevo-coronavirus-2019-ncov/casos-confirmados-en-chile-covid-19/'
response = get(url)

In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
#Almaceno los links de los archivos PDF
links = []
for table in html_soup.find_all('table'):
    for a in table.find_all('a'):
        links.append(a['href'])
    break

links

['https://www.minsal.cl/wp-content/uploads/2020/03/2020-03-14-Casos-confirmados.pdf',
 'https://www.minsal.cl/wp-content/uploads/2020/03/2020-03-13-Casos-Confirmados-Covid-19.pdf']

In [11]:
regiones = [] # [[región1, sexo1, edad1, centro_salud1] ....]
filas = [] # Se guarda cada una de las tuplas sin parsear aún

In [56]:
import requests

mypdf = requests.get(links[0])

In [57]:
import tabula

#Se extrae el pdf desde los links
df1 = tabula.read_pdf('https://www.minsal.cl/wp-content/uploads/2020/03/2020-03-14-Casos-confirmados.pdf', pages=1)

Got stderr: mar 14, 2020 8:39:39 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
mar 14, 2020 8:39:39 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored



In [74]:
df = df1[0]
len(df.index)

71

In [75]:
#Seteo columnas del DataFrame, se elimina la primera y última fila, por último se elimina la columna 'Casos confirmados'
df.columns = df.iloc[0].values
df = df.drop([0,len(df.index)-1]).reset_index(drop=True)
df = df.drop(['Casos confirmados'], axis=1)
df=df.rename(columns = {'Centro de salud':'Centro_salud'})
df.columns

Index(['Región', 'Sexo', 'Edad', 'Centro_salud'], dtype='object')

In [76]:
#Se eliminan filas que no tengan edad, se normaliza la columna sexo y Centro_salud, por último se reemplazan por valores que lo preceden
df = df[df["Edad"]!='—'].reset_index(drop=True)
df['Sexo'] = df['Sexo'].str.replace('Fememino','Femenino')
df['Centro_salud'] = df['Centro_salud'].str.replace('\r',' ')
df = df.fillna(method='ffill')

In [77]:
df

,Región,Sexo,Edad,Centro_salud
0,Antofagasta,Femenino,58,Hospital Carlos Cisternas Calama
1,Antofagasta,Masculino,63,Hospital Carlos Cisternas Calama
2,Atacama,Masculino,50,Hospital de Copiapó
3,Metropolitana,Femenino,54,Clínica Alemana
4,Metropolitana,Femenino,40,Clínica Santa María
5,Metropolitana,Masculino,58,Clínica Alemana
6,Metropolitana,Masculino,17,Clínica Alemana
7,Metropolitana,Femenino,83,Clínica Las Condes
8,Metropolitana,Masculino,14,Clínica Alemana
9,Metropolitana,Masculino,42,Clínica U. de Los Andes
